In [1]:
from os import listdir
from os.path import isfile, join
import json
from tqdm import tqdm
import pandas as pd
from copy import deepcopy

In [2]:
# Loading file list
path = '..'
files = listdir(path)
extension = '.json'
files = filter(lambda f: len(f) > len(extension), files)
files = map(lambda f: join(path, f), files)
files = list(filter(lambda pf: extension in pf and isfile(pf), files))

In [3]:
# Reading all files, storing data to rows
header = ['idx', 'name', 'eans', 'slug']
rows = []
for f in tqdm(files):
    product = json.load(open(f, 'r'))
    
    row = product['_product']['id'], product['_source']['name'], \
        product['_source']['eans'], product['_source']['categories'][0]['slug']
    for item in row:
        assert(len(item) > 0)
    
    eans_idx = header.index('eans')
    
    for ean in row[eans_idx]:
        r = list(deepcopy(row))
        r[eans_idx] = ean
        rows.append(r)

100%|██████████| 8294/8294 [00:04<00:00, 1923.01it/s]


In [4]:
# Creating a dataframe
df = pd.DataFrame(rows, columns = header)

In [5]:
df.head()

,idx,name,eans,slug
0,101146800000,V6 Dental White Spearmint,5700626503125,supermarche/denrees-alimentaires/chocolat/bonb...
1,101146800000,V6 Dental White Spearmint,5704592009001,supermarche/denrees-alimentaires/chocolat/bonb...
2,107306500000,M-Quick Boisson au cacao,7613312049297,supermarche/denrees-alimentaires/petit-dejeune...
3,107306500000,M-Quick Boisson au cacao,7610121500315,supermarche/denrees-alimentaires/petit-dejeune...
4,107306500000,M-Quick Boisson au cacao,7613312049303,supermarche/denrees-alimentaires/petit-dejeune...


In [6]:
# All of the categories
set([x.split('/')[2] for x in list(df['slug'])])

{'aperitif',
 'boissons-chaudes-froides',
 'boulangerie',
 'chocolat',
 'fruits-et-legumes',
 'garnitures-ingredients',
 'mueesli-cereales',
 'petit-dejeuner',
 'plats-cuisines',
 'poisson-fruits-de-mer',
 'produits-laitiers-oeufs',
 'produits-surgeles',
 'tartes',
 'viande'}

In [7]:
# Saving data to csv file
df.to_csv('migros-barcode-category.csv')